In [30]:
import numpy as np
import torch
import torch.nn as nn
import torch.onnx

In [31]:
class ModularMLP(nn.Module):
    def __init__(self, config):
        super(ModularMLP, self).__init__()
        self.input_size = config['input_size']
        self.layer_size = config['layer_size']
        self.num_hidden_layers = config['num_hidden_size']
        self.output_size = config['output_size']

        layers = []
        layers.append(nn.Linear(self.input_size, self.layer_size))
        layers.append(nn.ReLU())

        for _ in range(self.num_hidden_layers - 1):  # -1 because we want to add the final layer separately
            layers.append(nn.Linear(self.layer_size, self.layer_size))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(self.layer_size, self.output_size))

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

    def initialize_random_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight)
                nn.init.normal_(m.bias)

# Export the model to ONNX
def export_to_onnx(model, dummy_input, filename):
    torch.onnx.export(model, dummy_input, filename)

In [32]:
config = {
    'input_size': 200,
    'layer_size': 100,
    'num_hidden_size': 499,
    'output_size': 10
}

model = ModularMLP(config)
model.initialize_random_weights()  # Initialize with random weights

# Export the model
dummy_input = torch.randn(1, config['input_size'])  # Single input tensor of shape [1, 200]
export_to_onnx(model, dummy_input, f"mlp_{config['num_hidden_size']}.onnx")

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

